## 惰性赋值
摘抄自《scala程序设计（第2版）》————3.11 惰性赋值

**惰性赋值是与传名参数相关的技术,假如你希望以延迟的方式初始化某值,并且表达式不会被重复计算,则需要使用惰性赋值。**下面列举了一些需要用到该技术的常见场景。

* • 由于表达式执行代价昂贵(例如: 打开一个数据库连接), 因此我们希望能推迟该操作,直到我们确实需要表达式结果值时才执行它。  
* • 为了缩短模块的启动时间,可以将当前不需要的某些工作推迟执行。  
* • 为了确保对象中其他的字段的初始化过程能优先执行,需要将某些字段惰性化。我们会在 11.4 节讨论 Scala 对象模型时深入探讨这些场景。  

下面的示例中便应用了惰性赋值:
```scala
// src/main/scala/progscala2/rounding/lazy-init-val.sc

object ExpensiveResource {
  lazy val resource: Int = init()  
  def init(): Int = { 
    // do something expensive
    // 执行某些代价高昂的操作
    0
  }
}
```

lazy 关键字意味着求值过程将会被推迟,只有需要时才会执行计算。

那么惰性赋值与方法调用有那些差别呢?对于方法调用而言,每次调用方法时方法体都会被执行;而惰性赋值则不然,首次使用该值时,用于初始化的“代码体”才会被执行一次。这种只能执行一次的计算对于可变字段而言几乎没有任何意义。因此, **lazy 关键字并不能用于修饰 var 变量。**

我们通过保护式(guard)来实现惰性值。当客户代码引用了惰性值时,保护式会拦截引用并检查此时是否需要初始化惰性。由于保护式能确保惰性值在第一次访问之前便已初始化,因此增加保护式检查只有当第一次引用惰性值时才是必要的。但不幸的是,很难解除之后的惰性值保护式检查。所以,与“立刻”值相比,惰性值具有额外的开销。因此只有当保护式带来的额外开销小于初始化带来的开销时,或者将某些值惰性化(请参考11.4节)能简化系统初始化过程并确保执行顺序满足依赖条件时,你才应该使用惰性值。